<a href="https://colab.research.google.com/github/Praveenku32k/Agentic_CrewAI/blob/main/L2_research_write_article_Crew_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the rapid advancements and evolution of AI technologies

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning algorithms
- Natural language processing and chatbots
- Predictive analytics and AI-driven decision-making
- Automation and robotics in AI

2. Key Players in the AI Indus

I now can give a great answer

Final Answer:

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way businesses operate and interact with customers. The rapid advancements in AI technologies have paved the way for unprecedented levels of automation and intelligence in decision-making processes. As AI continues to evolve, it's crucial to stay updated on the latest trends and key players shaping the industry.

## Latest Trends in Artificial Intelligence

Machine learning and deep learning algorithms are at the forefront of AI innovation, enabling machines to learn from data and make predictions without explicit programming. Natural language processing (NLP) and chatbots have also gained significant traction, enhancing human-computer interactions and customer service experiences. Predictive analytics and AI-driven decision-making have become essential tools 

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way businesses operate and interact with customers. The rapid advancements in AI technologies have paved the way for unprecedented levels of automation and intelligence in decision-making processes. As AI continues to evolve, it's crucial to stay updated on the latest trends and key players shaping the industry.

## Latest Trends in Artificial Intelligence

Machine learning and deep learning algorithms are at the forefront of AI innovation, enabling machines to learn from data and make predictions without explicit programming. Natural language processing (NLP) and chatbots have also gained significant traction, enhancing human-computer interactions and customer service experiences. Predictive analytics and AI-driven decision-making have become essential tools for businesses looking to gain a competitive edge in their respective markets. Moreover, automation and robotics in AI are revolutionizing industries by streamlining processes and increasing efficiency.

## Key Players in the AI Industry

Several companies have emerged as key players in the AI landscape, driving innovation and pushing the boundaries of what is possible with artificial intelligence. Google DeepMind, IBM Watson, Amazon AI, Microsoft Cognitive Services, OpenAI, and NVIDIA are among the top contenders in the AI industry, each making significant contributions to the development and application of AI technologies.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research have led to groundbreaking applications of AI in healthcare, finance, and marketing, improving patient outcomes, optimizing financial strategies, and enhancing customer experiences. However, ethical considerations and regulations in AI development remain critical topics of discussion, as the potential implications of AI on society continue to evolve.

In conclusion, Artificial Intelligence is a dynamic and rapidly evolving field that holds immense potential for innovation and growth. By staying informed about the latest trends, key players, and ethical considerations in AI, individuals can actively participate in shaping the future of this transformative technology.

*Stay updated on Artificial Intelligence trends, engage with key players in the industry, and join the conversation on the impact of AI on society.*

Sources:
- [Forbes](https://www.forbes.com/artificial-intelligence/)
- [TechCrunch](https://techcrunch.com/tag/artificial-intelligence/)
- [MIT Technology Review](https://www.technologyreview.com/artificial-intelligence/)

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "CrewAI"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on CrewAI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: Unveiling the Power of CrewAI: A Comprehensive Guide

Introduction:
- Brief introduction to CrewAI and its significance in the current market
- Overview of the latest trends and key players in the CrewAI industry

Key Points:
1. Latest Trends in CrewAI:
- Integration of artificial intelligence and machine learning to enhance crew management processes
- Adoption of predictive analytics for optimized crew scheduling and resource allocation
- Implementation of real-time monitoring and communication

I now can give a great answer

Final Answer:
# Unveiling the Power of CrewAI: A Comprehensive Guide

In today's rapidly changing business landscape, effective crew management plays a vital role in the success of any organization. The emergence of CrewAI, which combines artificial intelligence and machine learning in crew management processes, has revolutionized the industry. Through the use of predictive analytics and real-time monitoring tools, CrewAI empowers organizations to optimize crew scheduling, resource allocation, and coordination, ultimately leading to enhanced productivity and performance.

### Latest Trends in CrewAI

The current trends in CrewAI are centered around enhancing the efficiency and effectiveness of crew management processes. By incorporating artificial intelligence and machine learning technologies, organizations can now automate and streamline crew scheduling and resource allocation tasks. The implementation of predictive analytics has transformed crew planni

In [ ]:
Markdown(result)

# Unveiling the Power of CrewAI: A Comprehensive Guide

In today's rapidly changing business landscape, effective crew management plays a vital role in the success of any organization. The emergence of CrewAI, which combines artificial intelligence and machine learning in crew management processes, has revolutionized the industry. Through the use of predictive analytics and real-time monitoring tools, CrewAI empowers organizations to optimize crew scheduling, resource allocation, and coordination, ultimately leading to enhanced productivity and performance.

### Latest Trends in CrewAI

The current trends in CrewAI are centered around enhancing the efficiency and effectiveness of crew management processes. By incorporating artificial intelligence and machine learning technologies, organizations can now automate and streamline crew scheduling and resource allocation tasks. The implementation of predictive analytics has transformed crew planning by offering insights into future demand and facilitating optimized crew assignments. Additionally, real-time monitoring and communication tools promote seamless coordination among crew members, resulting in improved operational efficiency and increased customer satisfaction.

### Key Players in the CrewAI Market

Several prominent players in the CrewAI market are at the forefront of reshaping crew management solutions. Crewsense, a leading provider of CrewAI solutions tailored for public safety organizations, offers innovative tools for efficient crew scheduling and resource allocation. Shiftboard specializes in workforce management solutions equipped with advanced CrewAI capabilities, allowing organizations to optimize their workforce and enhance operational efficiency. WhenToWork, recognized for its cloud-based employee scheduling software enhanced by AI, assists organizations in streamlining scheduling processes and improving crew coordination.

### Noteworthy News in CrewAI

Recent partnerships between CrewAI technology providers and major airlines have led to enhanced crew optimization and operational efficiency. The introduction of new CrewAI features and updates is aimed at boosting crew performance and productivity, setting new benchmarks in workforce management. Industry experts foresee a promising future for CrewAI in streamlining crew management processes and optimizing resource allocation to drive increased efficiency and cost savings.

In summary, CrewAI is reshaping the approach organizations take in managing their crews, with a strong emphasis on optimizing scheduling, resource allocation, and coordination. With the latest trends and key players in the market, CrewAI offers valuable solutions for enhancing operational efficiency and productivity. As the industry continues to progress, organizations stand to gain by exploring the advantages of CrewAI for their own workforce management requirements.

**Call to Action:** 
Discover the benefits of CrewAI for your organization and delve deeper into how it can transform your crew management processes. Click [here](#) to request a demo or trial of a CrewAI solution and witness firsthand the impact of AI in crew management.

*Resources:*
- [Crewsense website](www.crewsense.com)
- [Shiftboard website](www.shiftboard.com)
- [WhenToWork website](www.whentowork.com)
- Industry reports and articles on CrewAI advancements in crew management.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).